# **Selección de características en modelos predictivos**

## Inteligencia Artificial (IS) 2023/24 – Trabajo

### Autores: José María Baquero Rodríguez y Pedro Pablo Santos Domínguez

En este notebook se presenta una descripción detallada y exhaustiva en primer lugar se expone la implementación realizada para llevar a cabo el trabajo. 
Posteriormente, se muestran las pruebas realizadas utilizando los conjuntos de datos poporcionados.

## Importación de librerías y paquetes externos

Para implementar las funciones qeu se presentarán en este notebook, es necesrio importar una serie de bibliotecas y recursos. A continucación se detallan los elementos requeridos.

In [ ]:
#poner las importaciones
import pandas as pd

import numpy as np

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

from sklearn.tree import DecisionTreeClassifier

from matplotlib import pyplot
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer

Análisis de las importaciones:

- Numpy : Se utiliza para realizar cálculos científicos y será empleado en diversas ocasiones a lo largo de la implementación.
  
- Pandas : Es un paquete Python utilizado para el análisis de datos, lectura de archivos y tratamiento de información. En nuestro caso se utilizará para el procesamiento de los datos.

Además las siguientesimportaciones están relacionadas con el paquete  _scikit-learn_, que contienen funciones e implementaciones para el trabajo con aprendizaje automático en Python.

- cross_val_score : Proporcionada por  _sklearn_ permitiendo realizar un experimento de validación cruzada para obtener una tasa de acierto del algoritmo que le proporcionemos como parámetro.
  
-  DecisionTreeClassifier : Este algoritmo contruye árboles binarios y será evaluado mediante validación cruzada.



***

## Implementación

### Implemetación del método de evaluación

El algortimo _rendimiento_ evalúa el desempeño de un conjunto de característiacs en un modelo de decision utilizando validación cruzada. A continuación se resrnta unaexplicación paso a paso del algoritmo: 
 ---(seguir decripcion cuante este termionado con los 2 metodos)

In [ ]:
## codigo de la fucnion de rendimiento.
def rendimiento(atributos, objetivo, feature):
    
    # tree = decision_tree(atributos, objetivo, atributos_discretos)
    clasificador_CART = DecisionTreeClassifier()
    score = cross_validate(clasificador_CART, atributos_titanic[feature], objetivo_titanic, cv = 10, scoring = 'balanced_accuracy')
    return np.mean(score['test_score'])

### Implementación del algortimo de Busqueda secuelcial hacia atrás

El algoritmo de búsqueda secuencial hacia atrás es una técnica de selección de características que busca reducir el número de variables predictoras de un modelo, eliminando iterativamente las características menos importantes y evaluando el rendimiento del modelo en cada paso. Aquí se presenta una descripción detallada de su implementación:

- Descripción del Algoritmo

     - Entrada:
       - Conjunto de datos: Incluye N variables predictoras y una variable respuesta.
        - Parámetros adicionales: Número de experimentos (n_exp) y número de validaciones cruzadas (cv).
          
    - Salida:
        - Tabla de resultados: Contiene las combinaciones obtenidas en cada iteración, su tamaño y su rendimiento.
          
    - Inicialización:
        - Solución Actual (solucion_actual): Inicialmente contiene todas las variables predictoras disponibles.

    - Ejecución:
        - Iteración desde K = N hasta 1:
            - Para cada variable V en solucion_actual:
            Crear una Solución Temporal (solucion_temporal) eliminando la variable V de solucion_actual.
            Evaluar el rendimiento de solucion_temporal utilizando una evaluación robusta (validación cruzada) y guardar el rendimiento.
            - Seleccionar la Mejor Solución Temporal (la que proporcione el mayor rendimiento) y actualizar solucion_actual con esta mejor solución.
            - Guardar la Mejor Solución Temporal, su tamaño y su rendimiento en la tabla de resultados.
            - Repetir hasta que todas las variables hayan sido evaluadas y eliminadas una a una.
              
    - Devolución de Resultados:

        - La tabla de resultados contendrá K soluciones, cada una de diferente tamaño. La primera fila tendrá la solución con todas las variables y la última fila tendrá la solución con una sola variable.

In [34]:
## codigo de la busqueda
def busqueda_secuencial_hacia_atras(atributos, objetivo, n_exp=10, cv=10):
    # Inicialización
    solucion_actual = list(atributos.columns)
    n_features = len(solucion_actual)
    resultados = []

     # Guardar el rendimiento de la solución inicial completa
    rendimiento_inicial = rendimiento(atributos, objetivo, solucion_actual)
    resultados.append((solucion_actual.copy(), len(solucion_actual), rendimiento_inicial))

    
    # Ejecución
    while len(solucion_actual) > 1:
        
        mejor_rendimiento = -float('inf')
        peor_feature = None
        
        # Evaluar todas las posibles soluciones temporales
        for feature in solucion_actual:
            solucion_temporal = solucion_actual.copy()
            solucion_temporal.remove(feature)
            rendimiento_temporal = rendimiento(atributos, objetivo, solucion_temporal)

            print(f"Evaluando sin {feature}: {solucion_temporal} - Rendimiento: {rendimiento_temporal}")

            # Guardar el rendimiento de la peor característica
            if rendimiento_temporal > mejor_rendimiento:
                mejor_rendimiento = rendimiento_temporal
                peor_feature = feature
                
        
        # Actualizar la solución actual eliminando la peor característica
        solucion_actual.remove(peor_feature)

        resultados.append((solucion_actual.copy(), len(solucion_actual), mejor_rendimiento))
    
    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados, columns=['MejorSolucionTemporal', 'Tamaño', 'Rendimiento']).sort_values(by='Rendimiento', ascending=False)
    
    # Devolver los resultados en forma de DataFrame
    return df_resultados

### Implementación del algortimo de Busqueda secuencial hacia atrás mixta

El algoritmo de búsqueda secuencial hacia atrás mixta es una técnica avanzada de selección de características que combina estrategias de eliminación y adición de variables para encontrar el subconjunto óptimo de variables predictoras. Este método incorpora un control riguroso de las variables ya procesadas para evitar bucles infinitos y asegurar una evaluación robusta del rendimiento del modelo. A continuación, se presenta una descripción detallada de su implementación:

- Entrada:
    - Conjunto de datos: Incluye N variables predictoras y una variable respuesta.
    - Parámetros adicionales: Otros parámetros necesarios para la ejecución del algoritmo.

- Salida:
    - Tabla de resultados: Contiene las combinaciones obtenidas en cada iteración, su tamaño y su rendimiento.

- Inicialización:
    - SoluciónActual: Almacena el mejor conjunto de variables obtenido en cada iteración. Inicialmente contiene todas las variables.
    - Añadidos: Almacena las variables que han sido añadidas.
    - Eliminados: Almacena las variables que han sido eliminadas.

- Ejecución:
    - Mientras no se cumpla la CondiciónDeParada:

        - Eliminar la Peor Variable:

        - Seleccionar la peor variable para eliminar de SoluciónActual. Para cada variable V de SoluciónActual que no esté en Eliminados:
        - SoluciónTemporal = SoluciónActual - V
        - Evaluar SoluciónTemporal mediante una evaluación robusta y guardar su rendimiento.
        - Seleccionar la mejor SoluciónTemporal (la que proporcione mayor rendimiento) y actualizar SoluciónActual = MejorSoluciónTemporal.
        - Actualizar Eliminados añadiendo a esta lista la variable eliminada de SoluciónActual.
    - Añadir la Mejor Variable (Solo si Hay Mejora):

        - Seleccionar la mejor variable para añadir a SoluciónActual. Para cada variable V del conjunto original de variables que no se encuentre en SoluciónActual ni en Añadidos:
        - SoluciónTemporal = SoluciónActual + V
        - Evaluar SoluciónTemporal mediante una evaluación robusta y guardar su rendimiento.
        - Seleccionar la mejor SoluciónTemporal (la que proporcione mayor rendimiento). Solo si el rendimiento de la mejor SoluciónTemporal es superior al rendimiento de la mejor solución obtenida en el paso anterior:
        - Actualizar SoluciónActual = MejorSoluciónTemporal.
        - Actualizar Añadidos añadiendo la variable añadida.
        - Evaluar la Condición de Parada.
- Condición de Parada:
La condición de parada solo puede ser cierta una vez que todas las variables han sido procesadas por el proceso de eliminación, es decir, cuando Eliminados contiene todas las variables originales. En este punto, se utiliza un contador para contar las iteraciones que transcurren sin que se añadan variables (ya no se pueden eliminar variables). Si en alguna iteración una nueva variable es añadida, el contador se resetea a cero. Si el contador alcanza un umbral, se considera que se cumple la condición de parada y se detiene el algoritmo. Este criterio de parada permite obtener soluciones más robustas, asegurando que se han añadido todas las variables relevantes.

- Devolución:
Devolver una tabla con cada una de las MejorSoluciónTemporal (obtenida al final de cada iteración), el tamaño y el rendimiento de cada una.

In [5]:
## codigo de la busqueda
def busqueda_secuencial_hacia_atras_mixta(atributos, objetivo, n_exp=10, cv=7, umbral=10):
    # Inicialización
    solucion_actual = list(atributos.columns)
    añadidos = []
    eliminados = []
    iteraciones_sin_mejora = 0
    rendimiento_actual = rendimiento(atributos, objetivo, solucion_actual)
    
    resultados = [(solucion_actual.copy(), len(solucion_actual), rendimiento_actual)]
    
    # Ejecución
    while len(eliminados) < len(atributos.columns):
        mejor_rendimiento = -float('inf')
        peor_feature = None
        
        # Evaluar todas las posibles soluciones temporales eliminando una variable
        for feature in solucion_actual:
            if feature not in eliminados:
                solucion_temporal = solucion_actual.copy()
                solucion_temporal.remove(feature)
                rendimiento_temporal = rendimiento(atributos, objetivo, solucion_temporal)

                if rendimiento_temporal > mejor_rendimiento:
                    mejor_rendimiento = rendimiento_temporal
                    peor_feature = feature
        
        # Actualizar la solución actual eliminando la peor característica
        solucion_actual.remove(peor_feature)
        eliminados.append(peor_feature)
        rendimiento_actual = mejor_rendimiento

        resultados.append((solucion_actual.copy(), len(solucion_actual), rendimiento_actual))

        # Intentar añadir la mejor característica si hay mejora
        mejor_rendimiento_añadir = -float('inf')
        mejor_feature_añadir = None
        
        for feature in atributos.columns:
            if feature not in solucion_actual and feature not in añadidos:
                solucion_temporal = solucion_actual.copy()
                solucion_temporal.append(feature)
                rendimiento_temporal = rendimiento(atributos, objetivo, solucion_temporal)

                if rendimiento_temporal > mejor_rendimiento_añadir:
                    mejor_rendimiento_añadir = rendimiento_temporal
                    mejor_feature_añadir = feature

        if mejor_rendimiento_añadir > rendimiento_actual:
            solucion_actual.append(mejor_feature_añadir)
            añadidos.append(mejor_feature_añadir)
            rendimiento_actual = mejor_rendimiento_añadir
            iteraciones_sin_mejora = 0
        else:
            iteraciones_sin_mejora += 1

        resultados.append((solucion_actual.copy(), len(solucion_actual), rendimiento_actual))

        # Evaluar condición de parada
        if iteraciones_sin_mejora >= umbral:
            break
    
    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados, columns=['MejorSolucionTemporal', 'Tamaño', 'Rendimiento'])

    # Filtrar para que solo quede el mejor rendimiento por cada tamaño de conjunto
    df_resultados = df_resultados.sort_values(by=['Tamaño', 'Rendimiento'], ascending=[True, False])
    df_resultados = df_resultados.drop_duplicates(subset=['Tamaño'], keep='first')
    
    # Devolver los resultados en forma de DataFrame
    return df_resultados.sort_values(by=['Rendimiento'], ascending=False)

***

## Pruebas y experimentos

Para llevar a cabo las pruebas de los algoritmos y realizar experimentos, emplearemos varios conjuntos de datos suministrados por la propuesta de trabajo, los cuales ya están preprocesados.

Haremos uso de la librería Pandas, que nos permitirá tarbajar con los datos a través de DataFrames, los cuales servirán como base de información para los algoritmos que evaluaremos.

Inicialmente, cargaremos el conjunto de datos de los pasajeros del Titanic. Posteriormente, procederemos a importar el conjunto de datos relacionado con el cáncer de mama.


In [52]:
## lecturas de los 2 ficheros
def lectura_titanic():
    fichero = pd.read_csv('titanic.csv')    
    return fichero

def lectura_breastCancer():
    fichero = pd.read_csv('BreastCancer.csv')
    return fichero

titanic = lectura_titanic()
breast_cancer = lectura_breastCancer()

In [ ]:
# Tratamiento de los datos
# Eleccion de semilla inicial aleatoria.
np.random.seed(357823)

# Atributos discretos, continuos y objetivo de titanic.csv.
atributos_discretos_titanic = ['Sex', 'Embarked', 'Alone', 'Deck']
atributos_continuos_titanic = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 
                           'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married']
atributos_titanic = titanic.loc[:, atributos_discretos_titanic + atributos_continuos_titanic]
objetivo_titanic = titanic['Survived']

# Codificación de atributos discretos y objetivo 
codificador_atributos_discretos_titanic = OrdinalEncoder() 
atributos_titanic[atributos_discretos_titanic] = codificador_atributos_discretos_titanic.fit_transform(atributos_titanic[atributos_discretos_titanic])
codificador_objetivo_titanic = LabelEncoder() 
objetivo_titanic = codificador_objetivo_titanic.fit_transform(objetivo_titanic)

# Atributos discretos, continuos y objetivo de BreastCancer.csv.
atributos_continuos_breast_cancer = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 
                           'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 
                           'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error',
                           'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 
                           'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points',
                           'worst symmetry', 'worst fractal dimension']
																				
atributos_breast_cancer = breast_cancer.loc[:, atributos_continuos_breast_cancer]
objetivo_breast_cancer = breast_cancer['diagnosis']

# Codificación de atributos discretos y objetivo
codificador_objetivo_breast = LabelEncoder()
objetivo_breast_cancer = codificador_objetivo_breast.fit_transform(objetivo_breast_cancer)

### Pruebas básicas sobre los conjuntos de datos

Ejecutamos una estimación sobre los conjuntos de datos para comprobar su funcionamiento previo.

#### Conjunto de datos Titanic.csv

Primero probaremos el conjunto de caractterísticas completo, para  después ir reduciendo el número de características y comprobar sus resultados.

In [ ]:
# comprobar con todas las carac

In [45]:
# comprobar con 4 las carac

In [ ]:
# comprobar con 1  carac

#### Conjunto de datos BreastCancer.csv

Actuamos de igual forma, primero probamos con todas las características y vamos reduciento el número de las mismas para probar sus resultados.

In [ ]:
# comprobar con todas las carac

In [ ]:
# comprobar con 4 las carac

In [ ]:
# comprobar con 1  carac

(escribir alguna conclusion de las prubas)

### Pruebas Búsqueda secuencial hacia atrás

Ejecutamos el algoritmo de búsqueda secuencial hacia atrás sobre los distintos conjuntos de datos.

#### Conjunto de datos Titanic.csv

In [ ]:
# llamar funcion

poner conclusiones

#### Conjunto de datos BreastCancer.csv

In [ ]:
# llamar funcion

poner conclusiones

### Pruebas Búsqueda secuencial hacia atrás mixta

De forma analoga esta vez procedemos utilizando el algoritmo de búsqueda secuencial hacia atrás mixta sobre los conjuntos de datos.

#### Conjunto de datos Titanic.csv

In [ ]:
# llamaar funcion

conclusiones

#### Conjunto de datos BreastCancer.csv

In [ ]:
# llamaar funcion

conclusiones